###### Content under Creative Commons Attribution license CC-BY 4.0, code under BSD 3-Clause License © 2021 Adam Wickenheiser

## Lab 2 Simulator

This Python code is a simulation of the motor-driven mass-spring-damper experiment used in Lab 2.  This code will output a text file of data in the same format as output by the data acquisition software used in lab.

The following cells only need to be run once to load libraries and define functions.  We define the function `cart_forced_response`, to set up the given physical characteristics of the system as well as the equation of motion to govern the system.  This function is called by `solve_ivp` in a later cell, which is similar to MATLAB's `ode23` or `ode45`.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import matplotlib.patches as mpatches
from matplotlib.transforms import Affine2D
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

In [ ]:
def cart_forced_response(t,z,n_weights,damp_percent,KP,KI,KD):
    
    global I, e_prev
    
    # constants
    m = 0.3 + 0.49*n_weights
    b = 4 + 10*damp_percent
    k = 660.0
    mu = 0.02
    g = 9.81
    
    # derived system parameters
    A = b/m
    B = k/m
    K = 10000/m
    C = A*B - 0.35*K
    
    # error
    if t < dwell_time:
        r = float(commanded_pos)/160410    # convert counts to meters
    else:
        r = 0.0
    e = r - z[0]
    
    # feedback law
    I = I + e*dt
    deriv = (e-e_prev)/dt
    e_prev = e
    u_max = 10.0
    u = np.maximum(np.minimum(u_max,KP*e + KI/10*I + KD*500*deriv),-u_max)
    
    #zdot = [z[1], (b*z[1]+k*z[0]+mu*m*g*np.sign(z[1])-u)/(-m)]
    zdot = [z[1], z[2], -C*z[0]-B*z[1]-A*z[2]+K*u]
    
    return zdot

In the following cell, set the parameters of the experiment.  These are:

1. The commanded position of the cart, measured in encoder counts (keep this at 1000 for every simulation)
1. The dwell time, which is the period of the step-up (this should nominally be 3.0, but you may need to increase this by a few seconds if the cart is not reaching steady state before the step down)
1. The number of weights attached to the cart (keep this at 2 for every simulation)
1. The tightness of the air damper valve screw (keep this at 0.5 for every simulation)

Each time you change any of these parameters, you must re-run the cell.

In [ ]:
commanded_pos = 1000 # amplitude of step function [encoder counts]
dwell_time = 3.0     # duration of the step up [s]

n_weights = 2        # number of weights attached to the cart
damp_percent = 0.5   # tightness of the air damper valve screw

# controller gains
KP = 0.1             # proportional gain
KI = 0.0             # integral gain
KD = 0.0             # derivative gain

Run the following cell to run the simulation, plot the position vs. time graph, and create an animation of the cart's motion.  Note: this may take a few seconds to run; make sure you can observe the animation and plot before proceeding.

In [ ]:
t_span = [0.0, 2*dwell_time]
dt = 0.009
t_eval = np.arange(t_span[0],t_span[1],dt)
z0 = [0, 0, 0]
I = 0.0
e_prev = 0.0
sol = solve_ivp(cart_forced_response,t_span,z0,t_eval=t_eval,args=(n_weights,damp_percent,KP,KI,KD))

r = np.zeros_like(sol.t)
r[sol.t<dwell_time] = commanded_pos/160410*100
r[0] = 0
plt.figure(figsize=(10,5))
plt.plot(sol.t,100*sol.y[0,:],sol.t,r)
plt.ylabel('Cart position [cm]')
plt.xlabel('Time [s]')

fig, ax = plt.subplots() # You may need to adjust the figsize to fit your screen

# Draw the cart
w_cart = 12.7    # cart width [cm]
h_cart = 5.4     # cart height [cm]
d_wheel = 2.5    # wheel diameter [cm]

cart = mpatches.Rectangle([-w_cart/2,d_wheel],w_cart,h_cart,linewidth=1,edgecolor='k',facecolor='b')
ax.add_patch(cart)
left_wheel = mpatches.Circle((-w_cart/4,d_wheel/2),d_wheel/2,linewidth=1,edgecolor='k',facecolor='w')
ax.add_patch(left_wheel)
right_wheel = mpatches.Circle((w_cart/4,d_wheel/2),d_wheel/2,linewidth=1,edgecolor='k',facecolor='w')
ax.add_patch(right_wheel)
motor_input = mpatches.Rectangle([-15.1,d_wheel+h_cart/4],0.1,h_cart/2,linewidth=1,edgecolor='k',facecolor='k')
ax.add_patch(motor_input)
x_spring = np.linspace(0,1,501)
y_spring = np.sin((2*np.pi*10)*x_spring)
spring = plt.plot(x_spring,y_spring,color='k',linewidth=1)[0]
ax.add_line(spring)
plt.xlim((-16,16))
plt.ylim((0,12))

# Initialize the axes for animation
def init():
    ax.set_aspect('equal')
    return (cart,left_wheel,right_wheel,motor_input,spring)

    
# Update the position of the cart at time t_eval[i]
def update(i):
    x_cart = 100*sol.y[0,i]
    
    cart.set_transform(Affine2D().translate(x_cart,0) + ax.transData)
    left_wheel.set_transform(Affine2D().translate(x_cart,0) + ax.transData)
    right_wheel.set_transform(Affine2D().translate(x_cart,0) + ax.transData)
    spring.set_transform(Affine2D().scale(15-w_cart/2+x_cart,1) + Affine2D().translate(-15,d_wheel+h_cart/2) + ax.transData)
    return (cart,left_wheel,right_wheel,motor_input,spring)

ani = FuncAnimation(fig, update, frames=range(int(t_eval.size)), init_func=init, interval=dt, blit=True, repeat=False)
plt.close()
HTML(ani.to_jshtml(1/dt))

Run the following cell to save your data in a text file in the current folder.  This file will contain time and position ("Encoder 1 Pos") data points, as well as the commanded position "Commanded Pos" (the square wave reference trajectory).

In [ ]:
dataout = np.zeros((t_eval.size,6))
r = np.zeros_like(t_eval)
r[t_eval<dwell_time] = commanded_pos
dataout[:,0] = range(t_eval.size)
dataout[:,1] = sol.t
dataout[:,2] = r
dataout[:,3] = 160410*sol.y[0,:]

print('Enter file name for saving data (without extension):')
filename = input()+'.txt'
np.savetxt(filename,dataout,fmt=['%d','%2.3f','%d','%d','%d','%d'],delimiter='\t',header=' Sample   Time     Commanded Pos   Encoder 1 Pos   Encoder 2 Pos   Encoder 3 Pos \n')

Now, you have a data file that is identical to the in-lab hardware you can use to analyze the system.  You should repeat the simulation for the other trials by updating the parameters in the third cell from the top and running the subsequent cells.